In [ ]:
!pip install -q llama-index==0.10.57 openai==1.37.0 tiktoken==0.7.0 llama-index-tools-google==0.1.3 newspaper4k==0.9.3.1 lxml_html_clean

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.4 MB/s eta 0:00:00


In [ ]:
import os

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"
GOOGLE_SEARCH_KEY = "GOOGLE_SEARCH_KEY"
GOOGLE_SEARCH_ENGINE = "GOOGLE_SEARCH_ENGINE" # Search Engine ID

## LLM and Embedding Model

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

Settings.llm = OpenAI(model="gpt-4o-mini", temperature= 0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Using Agents/Tools


In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent


# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

# initialize ReAct agent
agent = ReActAgent.from_tools([multiply_tool], verbose=True)

In [ ]:
res = agent.chat("What is the multiplication of 43 and 45?")

# Final response
res.response

> Running step 1d3f5ada-78ed-4e7a-ae7a-c2fc5f152858. Step input: What is the multiplication of 43 and 45?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 43, 'b': 45}
Observation: 1935
> Running step 02f814ea-89fb-4b30-b41b-e7a262415a05. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The multiplication of 43 and 45 is 1935.


'The multiplication of 43 and 45 is 1935.'

## Define Google Search Tool


In [ ]:
from llama_index.tools.google import GoogleSearchToolSpec

tool_spec = GoogleSearchToolSpec(key=GOOGLE_SEARCH_KEY, engine=GOOGLE_SEARCH_ENGINE)

In [ ]:
# Import and initialize our tool spec
from llama_index.core.tools.tool_spec.load_and_search import LoadAndSearchToolSpec

# Wrap the google search tool to create an index on top of the returned Google search
wrapped_tool = LoadAndSearchToolSpec.from_defaults(
    tool_spec.to_tool_list()[0],
).to_tool_list()

## Create the Agent


In [ ]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(wrapped_tool, verbose=False)

In [ ]:
res = agent.chat("How many parameters LLaMA 3.2 model has?")

In [ ]:
res.response

'The LLaMA 3.2 model has the following parameters:\n- Vision LLMs: 11 billion and 90 billion parameters\n- Lightweight text-only models: 1 billion and 3 billion parameters.'

In [ ]:
res.sources

[ToolOutput(content='Content loaded! You can now search the information using read_google_search', tool_name='google_search', raw_input={'args': (), 'kwargs': {'query': 'LLaMA 3.2 model parameters'}}, raw_output='Content loaded! You can now search the information using read_google_search', is_error=False),
 ToolOutput(content='The LLaMA 3.2 model features small and medium-sized vision LLMs with 11 billion and 90 billion parameters, as well as lightweight text-only models with 1 billion and 3 billion parameters.', tool_name='read_google_search', raw_input={'args': (), 'kwargs': {'query': 'LLaMA 3.2 model parameters'}}, raw_output='The LLaMA 3.2 model features small and medium-sized vision LLMs with 11 billion and 90 billion parameters, as well as lightweight text-only models with 1 billion and 3 billion parameters.', is_error=False)]

# Using Tools w/ VectorStoreIndex


A limitation of the current agent/tool in LlamaIndex is that it **relies solely on the page description from the retrieved pages** to answer questions. This approach will miss answers that are not visible in the page's description tag. To address this, a possible workaround is to fetch the page results, extract the page content using the newspaper3k library, and then create an index based on the downloaded content. Also, the previous method stacks all retrieved items from the search engine into a single document, making it **difficult to pinpoint the exact source** of the response. However, the following method will enable us to present the sources easily.


## Define Google Search Tool


In [ ]:
from llama_index.tools.google import GoogleSearchToolSpec

tool_spec = GoogleSearchToolSpec(key=GOOGLE_SEARCH_KEY, engine=GOOGLE_SEARCH_ENGINE)

In [ ]:
search_results = tool_spec.google_search("LLaMA 3.2 model details")

In [ ]:
import json

search_results = json.loads(search_results[0].text)

## Read Each URL Contents


In [ ]:
import newspaper

pages_content = []

for item in search_results["items"]:

    try:
        article = newspaper.Article(item["link"])
        article.download()
        article.parse()
        if len(article.text) > 0:
            pages_content.append(
                {"url": item["link"], "text": article.text, "title": item["title"]}
            )
    except:
        continue

print(len(pages_content))

6


## Create the Index


In [ ]:
from llama_index.core import Document

# Convert the texts to Document objects so the LlamaIndex framework can process them.
documents = [
    Document(text=row["text"], metadata={"title": row["title"], "url": row["url"]})
    for row in pages_content
]

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

# Build index / generate embeddings using OpenAI.
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[SentenceSplitter(chunk_size=512, chunk_overlap=128)],
)

In [ ]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
query_engine = index.as_query_engine()

## Query


In [ ]:
response = query_engine.query("How many parameters LLaMA 3.2 model has? list exact sizes of the models")
print(response)

The Llama 3.2 models have the following parameter sizes: 1B, 3B, 11B, and 90B.


In [ ]:
# Show the retrieved nodes
for src in response.source_nodes:
    print("Title\t", src.metadata["title"])
    print("Source\t", src.metadata["url"])
    print("Score\t", src.score)
    print("-_" * 20)

Title	 Introducing Llama 3.2 models from Meta in Amazon Bedrock: A new ...
Source	 https://aws.amazon.com/blogs/aws/introducing-llama-3-2-models-from-meta-in-amazon-bedrock-a-new-generation-of-multimodal-vision-and-lightweight-models/
Score	 0.5448207816546468
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Title	 meta-llama/Llama-3.2-90B-Vision-Instruct - Demo - DeepInfra
Source	 https://deepinfra.com/meta-llama/Llama-3.2-90B-Vision-Instruct
Score	 0.5350659161754009
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
